In [1]:
import requests
import json
import time
import polars as pl    
import base64
import os
from tkinter import Tcl

In [2]:
def json_scraper(url, file_name):
    response = requests.request("GET", url)
    json_data = response.json()
    with open(file_name, "w", encoding="utf-8") as json_file:
        json.dump(json_data, json_file, ensure_ascii=False, indent=4)

base_directory = "//wsl.localhost/Ubuntu/home/harrison/airflow_proj/"

def nat_sort_dir(parent_directory):
    directory = os.listdir(parent_directory)
    list_of_files = Tcl().call('lsort', '-dict', directory)
    return list_of_files

In [3]:
def json_account_scraper_loop():
    pages_remaining = True
    page = 1
    current_json = base_directory + "account_pages/account_page_1.json"
    
    json_scraper("https://mainnet-idx.algonode.cloud/v2/assets/27165954/balances?currency-greater-than=6999", base_directory + "account_pages/account_page_1.json")
    
    
    while pages_remaining == True:
            time.sleep(0.5)
            file = open(current_json)
            data = json.load(file)
            
            data
            base_url = "https://mainnet-idx.algonode.cloud/v2/assets/27165954/balances&currency-greater-than=6999"
    
            page += 1
            new_url = base_url[:62] + "?next=" + data["next-token"] + "&currency-greater-than=6999"
            file.close()
            current_json = base_directory + "account_pages/account_page_" + str(page) + ".json"
    
            json_scraper(new_url, current_json)

In [4]:
def json_transaction_scraper_loop():
    pages_remaining = True
    page = 3203
    current_json = base_directory + "transaction_pages/transaction_page_3203.json"
    attempts = 0
    
    json_scraper("https://mainnet-idx.algonode.cloud/v2/transactions?next=FqStAQAAAAAYAAAA&tx-type=axfer&asset-id=27165954&after-time=2023-01-31T23%3A59%3A59-06%3A00&currency-greater-than=0&address=ZW3ISEHZUHPO7OZGMKLKIIMKVICOUDRCERI454I3DB2BH52HGLSO67W754&address-role=sender", base_directory + "transaction_pages/transaction_page_3203.json")
    
    
    while pages_remaining == True:
            file = open(current_json)
            data = json.load(file)
            
            data
            base_url = "https://mainnet-idx.algonode.cloud/v2/transactions?tx-type=axfer&asset-id=27165954&after-time=2023-01-31T23%3A59%3A59-06%3A00&currency-greater-than=0&address=ZW3ISEHZUHPO7OZGMKLKIIMKVICOUDRCERI454I3DB2BH52HGLSO67W754&address-role=sender"

            try:
                new_url = base_url[:50] + "?next=" + data["next-token"] + "&tx-type=axfer&asset-id=27165954&after-time=2023-01-31T23%3A59%3A59-06%3A00&currency-greater-than=0&address=ZW3ISEHZUHPO7OZGMKLKIIMKVICOUDRCERI454I3DB2BH52HGLSO67W754&address-role=sender"
                page += 1
                file.close()
                current_json = base_directory + "transaction_pages/transaction_page_" + str(page) + ".json"
                attempts = 0
            except:
                print(data)
                if attempts < 5:
                    time.sleep(5)
                    attempts += 1
                    file = open(base_directory + "transaction_pages/transaction_page_" + str(page - 1) + ".json")
                    data = json.load(file)
                    new_url = base_url[:50] + "?next=" + data["next-token"] + "&tx-type=axfer&asset-id=27165954&after-time=2023-01-31T23%3A59%3A59-06%3A00&currency-greater-than=0&address=ZW3ISEHZUHPO7OZGMKLKIIMKVICOUDRCERI454I3DB2BH52HGLSO67W754&address-role=sender"
                    file.close()
                    current_json = base_directory + "transaction_pages/transaction_page_" + str(page) + ".json"
                else:
                     break
            
            json_scraper(new_url, current_json)

In [10]:
def decoder(b64_string):
    decoded = json.loads(base64.b64decode(b64_string).decode("utf-8"))
    return decoded

def transaction_cleaning(file):
    transact_df = pl.read_json(file)
    time.sleep(0.01)
    transact = transact_df.drop("current-round", "next-token")
    transact = transact.explode("transactions")
    transact = transact.unnest("transactions")
    transact = transact.drop("closing-amount", "close-rewards", "fee", "first-valid", "confirmed-round", "genesis-hash", "genesis-id", "group", "intra-round-offset", "last-valid", "receiver-rewards", "round-time", "sender", "sender-rewards", "signature", "tx-type")
    transact = transact.unnest("asset-transfer-transaction")
    transact = transact.drop("asset-id", "close-amount")

    vals = [row["note"] for row in transact.iter_rows(named=True)]
    for i in vals:
        vals[vals.index(i)] = decoder(i)
    
    try:
        transact = transact.with_columns(pl.Series(name="decoded_notes", values=vals))
        transact = transact.unnest("decoded_notes")
        transact = transact.drop("note", "scale")
        transact = transact.rename({"id":"transactionId"})
        transact = transact.rename({"receiver":"accountAddress"})
        transact_cleaned = transact
        return transact_cleaned
    except:
        type_3_4_keys = {'date', 'streams', 'deviceId', 'type', 'qualifying', 'localScore', 'globalScore'}
        type_1_2_keys = {'date', 'streams', 'deviceId', 'type', 'status', 'tier', 'qualifying', 'localScore', 'globalScore', 'scale'}
        type_3_4_dev = []
        type_1_2_dev = []

        initial_type = vals[0].keys()

        if initial_type == type_3_4_keys:
            while i.keys() in vals == type_3_4_keys:
                type_3_4_dev.append(i)

            remaining = vals.index(type_3_4_dev[-1]) + 1
            
            for i in vals[remaining:]:
                type_1_2_dev.append(i)




def account_cleaning(file):
    accounts_df = pl.read_json(file)
    time.sleep(0.01)
    accounts_df = accounts_df.drop("current-round", "next-token")
    accounts_df = accounts_df.explode("balances")
    accounts_df = accounts_df.unnest("balances")
    accounts_df = accounts_df.rename({"address":"accountAddress"})
    accounts_df = accounts_df.rename({"amount":"balance"})
    try:
        accounts_df = accounts_df.drop("opted-in-at-round", "opted-out-at-round", "is-frozen", "deleted")
    except:
        accounts_df = accounts_df.drop("opted-in-at-round", "is-frozen", "deleted")
    return accounts_df


In [34]:
transact_df = pl.read_json(base_directory + "transaction_pages/" + "transaction_page_4110.json")
transact = transact_df.drop("current-round", "next-token")
transact = transact.explode("transactions")
transact = transact.unnest("transactions")
transact = transact.drop("closing-amount", "close-rewards", "fee", "first-valid", "confirmed-round", "genesis-hash", "genesis-id", "group", "intra-round-offset", "last-valid", "receiver-rewards", "round-time", "sender", "sender-rewards", "signature", "tx-type")
transact = transact.unnest("asset-transfer-transaction")
transact = transact.drop("asset-id", "close-amount")

vals = [row["note"] for row in transact.iter_rows(named=True)]
for i in vals:
    vals[vals.index(i)] = decoder(i)



In [62]:
type_3_4_keys = {'date', 'streams', 'deviceId', 'type', 'qualifying', 'localScore', 'globalScore'}
type_1_2_keys = {'date', 'streams', 'deviceId', 'type', 'status', 'tier', 'qualifying', 'localScore', 'globalScore', 'scale'}

type_3_4_dev = []
type_1_2_dev = []

count = 0

for i in vals:
    if i.keys() == type_3_4_keys:
        type_3_4_dev.append(i)
    else:
        type_1_2_dev.append(i)

type_3_4_dev

[{'date': '2023-03-06',
  'streams': 287,
  'deviceId': 'PW:KA_011670',
  'type': 3,
  'qualifying': True,
  'localScore': 348,
  'globalScore': 348},
 {'date': '2023-03-06',
  'streams': 287,
  'deviceId': 'PW:KA_011670',
  'type': 3,
  'qualifying': True,
  'localScore': 348,
  'globalScore': 348},
 {'date': '2023-03-06',
  'streams': 286,
  'deviceId': 'PW:KA_011669',
  'type': 3,
  'qualifying': True,
  'localScore': 260,
  'globalScore': 260},
 {'date': '2023-03-06',
  'streams': 286,
  'deviceId': 'PW:KA_011669',
  'type': 3,
  'qualifying': True,
  'localScore': 260,
  'globalScore': 260},
 {'date': '2023-03-06',
  'streams': 287,
  'deviceId': 'PW:KA_011547',
  'type': 3,
  'qualifying': True,
  'localScore': 383,
  'globalScore': 383},
 {'date': '2023-03-06',
  'streams': 287,
  'deviceId': 'PW:KA_011547',
  'type': 3,
  'qualifying': True,
  'localScore': 383,
  'globalScore': 383},
 {'date': '2023-03-06',
  'streams': 287,
  'deviceId': 'PW:KA_011765',
  'type': 3,
  'qualif

In [32]:
transact_jsons = []

broken_jsons = []

for i in nat_sort_dir(base_directory + "transaction_pages"):
    try:       
        cleaned_file = transaction_cleaning(base_directory + "transaction_pages/" + i)
        transact_jsons.append(cleaned_file)
    except:
        print(i)
        broken_jsons.append(i)
        continue
        #with open(base_directory + "transaction_pages_cleaned/transactions_cleaned.json", "w", encoding="utf-8") as json_file:
            #json.dump(jsons, json_file, ensure_ascii=False, indent=4)

transaction_page_4110.json
transaction_page_4111.json
transaction_page_4112.json
transaction_page_4125.json
transaction_page_4126.json
transaction_page_4140.json
transaction_page_4141.json
transaction_page_4142.json
transaction_page_4145.json
transaction_page_4147.json
transaction_page_4150.json
transaction_page_4152.json
transaction_page_4153.json
transaction_page_4154.json
transaction_page_4155.json
transaction_page_4156.json
transaction_page_4157.json
transaction_page_4158.json
transaction_page_4159.json
transaction_page_4160.json
transaction_page_4161.json
transaction_page_4162.json
transaction_page_4163.json
transaction_page_4164.json
transaction_page_4165.json
transaction_page_4166.json
transaction_page_4167.json
transaction_page_4168.json
transaction_page_4169.json
transaction_page_4170.json
transaction_page_4171.json
transaction_page_4172.json
transaction_page_4173.json
transaction_page_4174.json
transaction_page_4175.json
transaction_page_4176.json
transaction_page_4177.json
t

In [69]:
transact_jsons[-1]

amount,accountAddress,transactionId,date,streams,deviceId,type,status,tier,qualifying,localScore,globalScore
i64,str,str,str,i64,str,str,str,str,bool,i64,i64
33331200,"""LSAHUHHFNBWVSQ…","""5TQOYOF63JV2WN…","""2023-01-31""",96,"""PW:AQ_003303""","""1""","""L""","""1""",true,473,473
133324800,"""YKUNBCHE2XSARH…","""2UKCTT2GD7RU4C…","""2023-01-31""",96,"""PW:AQ_004575""","""1""","""L""","""1""",true,432,432
33331200,"""LSAHUHHFNBWVSQ…","""SB5FHXDBLG4KMO…","""2023-01-31""",96,"""PW:AQ_004575""","""1""","""L""","""1""",true,432,432
133324800,"""XQLAPNWA5D5CPD…","""UB7LQQUQFM6WXK…","""2023-01-31""",96,"""PW:AQ_000670""","""1""","""L""","""1""",true,569,569
33331200,"""LSAHUHHFNBWVSQ…","""V5ILDUASZPQI5F…","""2023-01-31""",96,"""PW:AQ_000670""","""1""","""L""","""1""",true,569,569
66662400,"""FNDSFHWXDUE7NQ…","""MCY5H2TNMU2P52…","""2023-01-31""",96,"""PW:AQ_006986""","""1""","""L""","""2""",true,276,276
16665600,"""LSAHUHHFNBWVSQ…","""BL5L6UFTLHD4ZS…","""2023-01-31""",96,"""PW:AQ_006986""","""1""","""L""","""2""",true,276,276
133324800,"""23GVKOWDTZZT3I…","""YBJVNZFDHCUUSH…","""2023-01-31""",96,"""PW:AQ_004341""","""1""","""L""","""1""",true,380,380
33331200,"""LSAHUHHFNBWVSQ…","""D5FQ636VB2UEFA…","""2023-01-31""",96,"""PW:AQ_004341""","""1""","""L""","""1""",true,380,380


In [37]:
combined_transact_df = pl.concat(transact_jsons, how="diagonal")
combined_transact_df

amount,accountAddress,transactionId,date,streams,deviceId,type,qualifying,localScore,globalScore,status,tier
i64,str,str,str,i64,str,str,bool,i64,i64,str,str
2090400,"""LSAHUHHFNBWVSQ…","""7NO45BZZ2UAYU2…","""2023-07-03""",52,"""PW:KA_017219""","""3""",false,315,315,null,null
964800,"""VU2OTSQUR2UUVU…","""ZNVVZ6VO5RW72P…","""2023-07-03""",6,"""PW:KA_003593""","""3""",false,508,508,null,null
241200,"""LSAHUHHFNBWVSQ…","""RNPBSOFHITOFMX…","""2023-07-03""",6,"""PW:KA_003593""","""3""",false,508,508,null,null
2572800,"""SPBQQFYHB36FQ3…","""WFDQFLIFLKR7H3…","""2023-07-03""",16,"""PW:KA_003704""","""3""",false,435,435,null,null
643200,"""LSAHUHHFNBWVSQ…","""LYLOJLOVGSFFMZ…","""2023-07-03""",16,"""PW:KA_003704""","""3""",false,435,435,null,null
8361600,"""ZUHERPCTH2DX4B…","""3GT2JISSUTERZA…","""2023-07-03""",52,"""PW:KA_003382""","""3""",false,414,414,null,null
2090400,"""LSAHUHHFNBWVSQ…","""FTVOUBD372TY6L…","""2023-07-03""",52,"""PW:KA_003382""","""3""",false,414,414,null,null
9969600,"""2ZKHSAZB7UTX46…","""KVOPNQDIW4DDT5…","""2023-07-03""",62,"""PW:KA_016266""","""3""",false,455,455,null,null
2492400,"""LSAHUHHFNBWVSQ…","""OF5VAYXGKJXMJJ…","""2023-07-03""",62,"""PW:KA_016266""","""3""",false,455,455,null,null


In [43]:
account_jsons = []

for i in os.listdir(base_directory + "account_pages"):
    cleaned_file = account_cleaning(base_directory + "account_pages/" + i)
    account_jsons.append(cleaned_file)

In [44]:
combined_account_df = pl.concat(account_jsons, how="vertical")
combined_account_df

accountAddress,balance
str,i64
"""445DKUHTNTU5H2…",15994667949
"""445H43QIPJO65N…",1376256000
"""4454BE6FK3JYTN…",174231811750
"""4454ITEEBWBDEC…",9198179200
"""446SPBJR5NJSDC…",10000
"""446S3AEXQDLUWF…",377585000
"""447YPTKGPPTEZV…",6612634429
"""4477P7VSZ2733U…",10000
"""45AQAPCN233P6A…",129295416


In [46]:
matching_accounts_df = combined_account_df.join(combined_transact_df, on="accountAddress", how="inner")
matching_accounts_df

accountAddress,balance,amount,transactionId,date,streams,deviceId,type,qualifying,localScore,globalScore,status,tier
str,i64,i64,str,str,i64,str,str,bool,i64,i64,str,str
"""LSAHUHHFNBWVSQ…",3885799128729,2090400,"""7NO45BZZ2UAYU2…","""2023-07-03""",52,"""PW:KA_017219""","""3""",false,315,315,null,null
"""VU2OTSQUR2UUVU…",123958821749,964800,"""ZNVVZ6VO5RW72P…","""2023-07-03""",6,"""PW:KA_003593""","""3""",false,508,508,null,null
"""LSAHUHHFNBWVSQ…",3885799128729,241200,"""RNPBSOFHITOFMX…","""2023-07-03""",6,"""PW:KA_003593""","""3""",false,508,508,null,null
"""SPBQQFYHB36FQ3…",2239029960,2572800,"""WFDQFLIFLKR7H3…","""2023-07-03""",16,"""PW:KA_003704""","""3""",false,435,435,null,null
"""LSAHUHHFNBWVSQ…",3885799128729,643200,"""LYLOJLOVGSFFMZ…","""2023-07-03""",16,"""PW:KA_003704""","""3""",false,435,435,null,null
"""ZUHERPCTH2DX4B…",5193518400,8361600,"""3GT2JISSUTERZA…","""2023-07-03""",52,"""PW:KA_003382""","""3""",false,414,414,null,null
"""LSAHUHHFNBWVSQ…",3885799128729,2090400,"""FTVOUBD372TY6L…","""2023-07-03""",52,"""PW:KA_003382""","""3""",false,414,414,null,null
"""2ZKHSAZB7UTX46…",25117618719,9969600,"""KVOPNQDIW4DDT5…","""2023-07-03""",62,"""PW:KA_016266""","""3""",false,455,455,null,null
"""LSAHUHHFNBWVSQ…",3885799128729,2492400,"""OF5VAYXGKJXMJJ…","""2023-07-03""",62,"""PW:KA_016266""","""3""",false,455,455,null,null
